# LDL and Cholesky factorizations

## Symmetric elimination

Suppose we have a symmetric (or hermitian) matrix $A$. We'd like to symmetrize the $A=LU$ factorization. We can't simply set $U=L^*$, because $L$ has ones on the diagonal. But the factorization $A=LDL^*$ will work for diagonal D and unit lower triangular $L$. 

Defining (in the real case)

$$A_j = \sum_{k=j}^m d_k \ell_k \ell_k^T,$$

we have $A_1e_1=Ae_1=d_1 \ell_1$. Thanks to the normalization $L_{1,1}=1$, we can deduce $d_1$ and $\ell_1$ from this. 

In [1]:
using LinearAlgebra

A = [ 
    33     7    12    17
     7    23    17    22
    12    17    13    27
    17    22    27     3
];

L = LowerTriangular(zeros(4,4))
D = Diagonal(zeros(4,4))

A1 = A
D[1,1] = A1[1,1]
L[:,1] = A1[:,1]/D[1,1]
display(D)
display(L)

4×4 Diagonal{Float64,Array{Float64,1}}:
 33.0   ⋅    ⋅    ⋅ 
   ⋅   0.0   ⋅    ⋅ 
   ⋅    ⋅   0.0   ⋅ 
   ⋅    ⋅    ⋅   0.0

4×4 LowerTriangular{Float64,Array{Float64,2}}:
 1.0        ⋅    ⋅    ⋅ 
 0.212121  0.0   ⋅    ⋅ 
 0.363636  0.0  0.0   ⋅ 
 0.515152  0.0  0.0  0.0

Things unfold much as in basic $LU$. The next step is to get $A_2$, which is zero in the first row and column. Then $A_2e_2=d_2\ell_2$, and so on.

In [2]:
A2 = A1 - D[1,1]*L[:,1]*L[:,1]'

4×4 Array{Float64,2}:
 0.0   0.0      0.0       0.0    
 0.0  21.5152  14.4545   18.3939 
 0.0  14.4545   8.63636  20.8182 
 0.0  18.3939  20.8182   -5.75758

In [3]:
D[2,2] = A2[2,2]
L[:,2] = A2[:,2]/D[2,2]
display(D)
display(L)

4×4 Diagonal{Float64,Array{Float64,1}}:
 33.0    ⋅       ⋅    ⋅ 
   ⋅   21.5152   ⋅    ⋅ 
   ⋅     ⋅      0.0   ⋅ 
   ⋅     ⋅       ⋅   0.0

4×4 LowerTriangular{Float64,Array{Float64,2}}:
 1.0        ⋅         ⋅    ⋅ 
 0.212121  1.0        ⋅    ⋅ 
 0.363636  0.671831  0.0   ⋅ 
 0.515152  0.85493   0.0  0.0

In [4]:
A3 = A2 - D[2,2]*L[:,2]*L[:,2]'

4×4 Array{Float64,2}:
 0.0  0.0   0.0        0.0    
 0.0  0.0   0.0        0.0    
 0.0  0.0  -1.07465    8.46056
 0.0  0.0   8.46056  -21.4831 

This result is symmetric, which is no accident. In fact we can get here simply by

In [5]:
D[3,3] = A3[3,3]
L[:,3] = A3[:,3]/D[3,3]
A4 = A3 - D[3,3]*L[:,3]*L[:,3]'
D[4,4] = A4[4,4]
L[4,4] = 1

display(D)
display(L)

4×4 Diagonal{Float64,Array{Float64,1}}:
 33.0    ⋅        ⋅         ⋅    
   ⋅   21.5152    ⋅         ⋅    
   ⋅     ⋅      -1.07465    ⋅    
   ⋅     ⋅        ⋅       45.1258

4×4 LowerTriangular{Float64,Array{Float64,2}}:
 1.0        ⋅          ⋅        ⋅ 
 0.212121  1.0         ⋅        ⋅ 
 0.363636  0.671831   1.0       ⋅ 
 0.515152  0.85493   -7.87287  1.0

As long as we apply elimination symmetrically on the left and right, we preserve the symmetry of $A$ as the elimination proceeds. 

In [6]:
norm(A - L*D*L')

0.0

Like unpivoted LU factorization, the $A=LDL^*$ factorization is unstable. It can be pivoted into stability, but the details are surprisingly subtle, and it's not used often. We're after something more important.

## HPD matrices and the Cholesky factorization

If in addition to symmetry/hermitianness we also know that the quadratic form $x^*Ax$ is positive for all nonzero $x$, then $A$ is said to be **positive definite** (often written **SPD** or **HPD**). By taking $x=e_k$, we can see that $a_{kk}>0$; i.e., all diagonal entries are positive. 

Let's rerun the symmetric factorization, with a matrix guaranteed to be HPD.

In [7]:
A = A'*A/100

4×4 Array{Float64,2}:
 15.71   9.7   11.3   10.9 
  9.7   13.51  12.9   11.5 
 11.3   12.9   13.31  10.1 
 10.9   11.5   10.1   15.11

In the first step of the symmetric factorization, we isolate $d_1\ell_1\ell_1^*$. But we know that $d_1=1/A_{1,1}>0$, so we can write this as $(c_1\ell_1)(c_1\ell_1)^*$, where $c_1^2=d_1$. Define $s_1=c_1\ell_1$. This gives us $A_1e_1 = c_1s_1$. 

In [8]:
S = LowerTriangular(zeros(4,4))

c1 = sqrt(A[1,1])
S[:,1] = A[:,1]/c1
display(S)

4×4 LowerTriangular{Float64,Array{Float64,2}}:
 3.96358   ⋅    ⋅    ⋅ 
 2.44728  0.0   ⋅    ⋅ 
 2.85095  0.0  0.0   ⋅ 
 2.75004  0.0  0.0  0.0

Now we proceed to $A_2$. 

In [9]:
A2 = A - S[:,1]*S[:,1]'

4×4 Array{Float64,2}:
 1.77636e-15  1.77636e-15  0.0      1.77636e-15
 1.77636e-15  7.52082      5.92292  4.76989    
 0.0          5.92292      5.18206  2.25977    
 1.77636e-15  4.76989      2.25977  7.5473     

We want to proceed recursively, so it's fortunate that the (2,2) element here is positive. In general, that lower-right submatrix is positive definite, as one can prove using an argument similar to the book's theorem. Assuming that's in hand, we know that we can continue.

In [10]:
c2 = sqrt(A2[2,2])
S[2:4,2] = A2[2:4,2]/c2
display(S)

4×4 LowerTriangular{Float64,Array{Float64,2}}:
 3.96358   ⋅        ⋅    ⋅ 
 2.44728  2.74241   ⋅    ⋅ 
 2.85095  2.15975  0.0   ⋅ 
 2.75004  1.73931  0.0  0.0

In [11]:
A3 = A2 - S[:,2]*S[:,2]'

4×4 Array{Float64,2}:
 1.77636e-15   1.77636e-15   0.0        1.77636e-15
 1.77636e-15  -8.88178e-16   0.0       -8.88178e-16
 0.0           0.0           0.517549  -1.49669    
 1.77636e-15  -8.88178e-16  -1.49669    4.52212    

In [12]:
c3 = sqrt(A3[3,3])
S[3:4,3] = A3[3:4,3]/c3
display(S)

4×4 LowerTriangular{Float64,Array{Float64,2}}:
 3.96358   ⋅         ⋅         ⋅ 
 2.44728  2.74241    ⋅         ⋅ 
 2.85095  2.15975   0.719408   ⋅ 
 2.75004  1.73931  -2.08044   0.0

In [13]:
A4 = A3 - S[:,3]*S[:,3]'

4×4 Array{Float64,2}:
 1.77636e-15   1.77636e-15   0.0           1.77636e-15
 1.77636e-15  -8.88178e-16   0.0          -8.88178e-16
 0.0           0.0          -1.11022e-16   4.44089e-16
 1.77636e-15  -8.88178e-16   4.44089e-16   0.193868   

In [14]:
S[4,4] = sqrt(A4[4,4])
display(S)

4×4 LowerTriangular{Float64,Array{Float64,2}}:
 3.96358   ⋅         ⋅         ⋅      
 2.44728  2.74241    ⋅         ⋅      
 2.85095  2.15975   0.719408   ⋅      
 2.75004  1.73931  -2.08044   0.440305

In [15]:
norm(A-S*S')

4.351167857633658e-15

We have factored $A=SS^*$ for lower triangular $S$. This is the **Cholesky factorization**. It is usually written in terms of an upper triangular matrix, $A=R^*R$. 

In [16]:
cholesky(A)

Cholesky{Float64,Array{Float64,2}}
U factor:
4×4 UpperTriangular{Float64,Array{Float64,2}}:
 3.96358  2.44728  2.85095    2.75004 
  ⋅       2.74241  2.15975    1.73931 
  ⋅        ⋅       0.719408  -2.08044 
  ⋅        ⋅        ⋅         0.440305

In a nice wrinkle, Cholesky factorization is stable out of the box---no pivoting is ever necessary.